# Import modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
import datetime

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# ex: from src.create_model_pickle import *


# Import data

In [4]:
# Use our json to dataframe conversion class
from src.pipeline import pipeline_json
pj = pipeline_json('../data/data.json')

In [43]:
df = pj.convert_to_df()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14337 entries, 0 to 14336
Data columns (total 53 columns):
acct_type              14337 non-null object
approx_payout_date     14337 non-null datetime64[ns]
body_length            14337 non-null int64
channels               14337 non-null int64
country                14256 non-null object
currency               14337 non-null object
delivery_method        14321 non-null float64
description            14337 non-null object
email_domain           14337 non-null object
event_created          14337 non-null datetime64[ns]
event_end              14337 non-null datetime64[ns]
event_published        14238 non-null datetime64[ns]
event_start            14337 non-null datetime64[ns]
fb_published           14337 non-null bool
gts                    14337 non-null float64
has_analytics          14337 non-null bool
has_header             14337 non-null float64
has_logo               14337 non-null bool
listed                 14337 non-null int64
na

date_cols = ['approx_payout_date', 'event_created', 'event_created', 'event_end', \
                     'event_published', 'event_start', 'user_created']

for col in date_cols:
    print df[col][:5] - df['user_created'][:5]

In [ ]:
df['has_header'][:10]
    

a = df['event_published'][:5] - df['user_created'][:5]
a.var

i=12
total_payout = 0
payout_count = len(df['previous_payouts'][i])
payees = set()
for d in df['previous_payouts'][i]:
    total_payout += d['amount']
    payees.add(d['name'])
print total_payout, payout_count, payees

In [ ]:
a = set()

In [16]:
df['ticket_types'][0][0].keys()

[u'event_id', u'cost', u'availability', u'quantity_total', u'quantity_sold']

In [17]:
Counter(df['fraud'])

Counter({False: 13044, True: 1293})

In [36]:
col = 'ticket_sales_amount'
print col
print "fraud :", df[df['fraud'] == True][col].mean()
print "not fraud :", df[df['fraud'] == False][col].mean()

ticket_sales_count
fraud : 1.73008507347
not fraud : 2.78994173566


In [32]:
col = 'total_payout'
print col
print "fraud :", df[df['fraud'] == True][col].mean()
print "not fraud :", df[df['fraud'] == False][col].mean()

total_payout
fraud : 1227.46851508
not fraud : 93904.2333517
